In [2]:
#dataframes department with total and month>> take around 3mins30sec
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pymongo import MongoClient
from collections import OrderedDict
from pyspark.sql.functions import *
import datetime
import sys

client = MongoClient("mongodb://192.168.187.129:27017")
db = client.hrvisual

conf = SparkConf().setAppName("building department_total_to_mongodb")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

#strYear='2015'
strYear=sys.argv[1]

collectionName = 'LEAVE_TYPE_%s' % strYear
#if the collection exises in mongodb, then drop it
#if collName in db.collection_names():
#        db[collName].drop()            

def getRddWithoutHeader(csvfname, fieldname):
    f  = sc.textFile(csvfname)
    r = f.filter(lambda l: fieldname in l)
    r.collect()
    r1 = f.subtract(r)
    d=r1.map(lambda k: k.split(","))
    return d

arrayferialname=["特別休假","加班或假日出差轉補休","生理假","傷病假","婚假","家庭照顧假", \
                    "事假","產檢假","陪產假","產假","喪假","國內公假","國外公假","公傷病假", "安胎假"]

def getFerialDict(fList, start):
    mydic={}
    for i in range(0,len(arrayferialname),1):
        mydic[arrayferialname[i]]=int(fList[i+start])
    
    return mydic

#build up schema
structFields=[StructField("dept_id", StringType()), StructField("dept_name", StringType()), StructField("month", IntegerType())]
for i in range(1,len(arrayferialname) +1,1):
    structFields.append(StructField(arrayferialname[i-1], IntegerType()))

schema = StructType(structFields)

print "===========read csv data============", datetime.datetime.now()
data=getRddWithoutHeader('LEAVE_TYPE_%s.csv' % strYear,'dept_id')
lines_temp = data.map(lambda p: (str(p[2]),unicode(p[3]),int(p[10]),int(p[11]),int(p[12]),int(p[13]),int(p[14]),int(p[15]), \
    int(p[16]), int(p[17]),int(p[18]),int(p[19]),int(p[20]),int(p[21]),int(p[22]),int(p[23]),int(p[24]),int(p[25])))


lines_df = sqlContext.createDataFrame(lines_temp, schema)
#print lines_df.printSchema()
print "============group by dept_id / month data============"
line_group=lines_df.groupBy("dept_id", "dept_name").sum('特別休假','加班或假日出差轉補休','生理假','傷病假','婚假','家庭照顧假', \
            '事假','產檢假', '陪產假','產假','喪假','國內公假','國外公假','公傷病假', '安胎假')

line_group_month=lines_df.groupBy("dept_id", "dept_name", "month").sum('特別休假','加班或假日出差轉補休','生理假','傷病假','婚假', \
            '家庭照顧假', '事假','產檢假', '陪產假','產假','喪假','國內公假','國外公假','公傷病假', '安胎假')

line_sort=line_group.sort(asc("dept_id")).collect()
print "============preparing list for mongodb============"
mylist = []
for a in line_sort: 
    rowDb={}
    rowDb["dept_id"]=a[0]
    rowDb["dept_name"]=a[1]
    rowDb["Total"]=getFerialDict(a,2)
    line_filter=line_group_month.filter(line_group_month.dept_id==a[0]).collect()
    for b in line_filter:
        rowDb[str(b[2])]=getFerialDict(b,3)
        
    rowDb=OrderedDict(sorted(rowDb.items(), key=lambda t: t[0]))
    mylist.append(rowDb)
    print 'list size>>',len(mylist)   
    
print "============start insert organzation statistics into mongodb============"
result=db[collectionName].insert_many(mylist)       
print "============finish to insert org total into mongodb============", datetime.datetime.now()

===========read csv data============ 2016-05-12 03:46:15.414388
============group by dept_id / month data============
============preparing list for mongodb============
list size>> 1
list size>> 2
list size>> 3
list size>> 4
list size>> 5
list size>> 6
list size>> 7
list size>> 8
list size>> 9
list size>> 10
list size>> 11
list size>> 12
list size>> 13
list size>> 14
list size>> 15
list size>> 16
list size>> 17
list size>> 18
list size>> 19
list size>> 20
list size>> 21
list size>> 22
list size>> 23
list size>> 24
list size>> 25
list size>> 26
list size>> 27
list size>> 28
list size>> 29
list size>> 30
list size>> 31
list size>> 32
list size>> 33
list size>> 34
list size>> 35
list size>> 36
list size>> 37
list size>> 38
list size>> 39
list size>> 40
list size>> 41
list size>> 42
list size>> 43
list size>> 44
list size>> 45
list size>> 46
list size>> 47
list size>> 48
list size>> 49
list size>> 50
list size>> 51
list size>> 52
list size>> 53
list size>> 54
list size>> 55
list size>> 56


In [2]:
#dataframes
from pyspark import SparkContext
from pyspark.sql.types import *
from pymongo import MongoClient
from collections import OrderedDict
from pyspark.sql.functions import *

client = MongoClient("mongodb://192.168.187.128:27017")
db = client.hrvisual

sqlContext = SQLContext(sc)

strYear='2015'

collectionName = 'LEAVE_TYPE_%s' % strYear
#if the collection exises in mongodb, then drop it
#if collName in db.collection_names():
#        db[collName].drop()            

def getRddWithoutHeader(csvfname, fieldname):
    f  = sc.textFile(csvfname)
    r = f.filter(lambda l: fieldname in l)
    r.collect()
    r1 = f.subtract(r)
    d=r1.map(lambda k: k.split(","))
    return d

arrayferialname=["特別休假","加班或假日出差轉補休","生理假","傷病假","婚假","家庭照顧假", \
                    "事假","產檢假","陪產假","產假","喪假","國內公假","國外公假","公傷病假", "安胎假"]

#build up schema
structFields=[StructField("dept_id", StringType()), StructField("dept_name", StringType())]
for i in range(1,len(arrayferialname) +1,1):
    structFields.append(StructField(arrayferialname[i-1], IntegerType()))

schema = StructType(structFields)


data=getRddWithoutHeader('LEAVE_TYPE_%s.csv' % strYear,'dept_id')
lines_temp = data.map(lambda p: (str(p[2]),unicode(p[3]),int(p[11]),int(p[12]),int(p[13]),int(p[14]),int(p[15]),int(p[16]), \
    int(p[17]),int(p[18]),int(p[19]),int(p[20]),int(p[21]),int(p[22]),int(p[23]),int(p[24]),int(p[25])))


lines_df = sqlContext.createDataFrame(lines_temp, schema)
#print lines_df.printSchema()

line_group=lines_df.groupBy("dept_id", "dept_name").sum('特別休假','加班或假日出差轉補休','生理假','傷病假','婚假','家庭照顧假', \
            '事假','產檢假', '陪產假','產假','喪假','國內公假','國外公假','公傷病假', '安胎假')

#insert into mongodb
line_sort=line_group.sort(asc("dept_id")).collect()
for a in line_sort: 
    rowDb={}
    rowDb["dept_id"]=a[0]
    rowDb["dept_name"]=a[1]
    
    sumArray={}
    for i in range(0,len(arrayferialname),1):
        sumArray[arrayferialname[i]]=int(a[i+2])
    
    rowDb["Total"]=sumArray
    rowDb=OrderedDict(sorted(rowDb.items(), key=lambda t: t[0]))
    #print 'rowDb=', rowDb
    result=db[collectionName].insert_one(rowDb)
    

print "finish to insert into mongodb"

finish to insert into mongodb


In [5]:
#dataframes
from pyspark import SparkContext
from pyspark.sql.types import *
from pymongo import MongoClient
from collections import OrderedDict
from pyspark.sql.functions import *

client = MongoClient("mongodb://192.168.187.128:27017")
db = client.hrvisual

sqlContext = SQLContext(sc)

strYear='2015'

collectionName = 'TEST_%s' % strYear
#if the collection exises in mongodb, then drop it
if collectionName in db.collection_names():
        db[collectionName].drop()            

json={
    "org_id" : "91",
    "org_name" : "財團法人國家實驗研究院國家災害防救科技中心",
    "Total" : {
        "公傷病假" : 0,
        "產假" : 0,
        "傷病假" : 0,
        "安胎假" : 0,
        "產檢假" : 0,
        "加班或假日出差轉補休" : 0,
        "婚假" : 0,
        "國內公假" : 0,
        "陪產假" : 0,
        "特別休假" : 0,
        "生理假" : 0,
        "家庭照顧假" : 0,
        "國外公假" : 0,
        "事假" : 0,
        "喪假" : 0
        }
}
result=db[collectionName].insert_one(json)
    

print "finish to insert into mongodb"

finish to insert into mongodb
